In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import os

# 1. 데이터 불러오기
# 파일 경로를 본인의 환경에 맞게 수정하세요 (예: 'news_data.csv' 또는 'news_data.xlsx')
df = pd.read_csv('data/yna_news.csv')

print(f"데이터 로드 완료: 총 {len(df)}건")

# 2. 전처리 준비 (형태소 분석기)
okt = Okt()

def tokenizer(text):
    # 명사만 추출하고, 2글자 이상이며 숫자가 아닌 것만 필터링
    return [n for n in okt.nouns(str(text)) if len(n) > 1 and not n.isdigit()]

# 3. 산업별로 기사 본문(article) 합치기
# TF-IDF는 '문서' 단위로 계산하므로, 산업별로 모든 기사를 합쳐 하나의 거대 문서를 만듭니다.
print("산업별 데이터 그룹화 중...")
industry_docs = df.groupby('산업')['article'].apply(lambda x: ' '.join(str(v) for v in x)).reset_index()

# 4. TF-IDF 벡터화 실행
print("TF-IDF 계산 및 키워드 추출 중 (이 과정은 사양에 따라 수초~수분이 소요될 수 있습니다)...")
vectorizer = TfidfVectorizer(
    tokenizer=tokenizer,
    max_df=0.7,      # 70% 이상의 산업에서 공통으로 나오는 단어 자동 제거 (불용어 사전 대체)
    min_df=1,        # 최소 1개 이상의 산업에서 나타나는 단어 유지
    max_features=100000 # 메모리 효율과 핵심어 집중을 위해 상위 3000개 단어만 사용
)

tfidf_matrix = vectorizer.fit_transform(industry_docs['article'])
feature_names = vectorizer.get_feature_names_out()

# 5. 결과 정리 및 출력
industry_keywords = {}

for i, row in industry_docs.iterrows():
    industry_name = row['산업']
    # 해당 산업 벡터에서 TF-IDF 점수가 높은 순으로 정렬
    row_data = tfidf_matrix.getrow(i).toarray()[0]
    
    # 상위 30개 핵심어 추출로 변경 ( -30 )
    top_indices = row_data.argsort()[-30:][::-1] 
    
    keywords = [feature_names[idx] for idx in top_indices]
    industry_keywords[industry_name] = keywords

# 최종 결과 출력 (상위 50개 전체 출력)
print("\n" + "="*30)
print("산업별 핵심 키워드 추출 결과 (Top 30)")
print("="*30)
for industry, keywords in industry_keywords.items():
    # 출력 부분도 30개 전체가 보이도록 [:50]으로 변경
    print(f"[{industry}]:\n{', '.join(keywords[:30])}\n")

데이터 로드 완료: 총 869건
산업별 데이터 그룹화 중...
TF-IDF 계산 및 키워드 추출 중 (이 과정은 사양에 따라 수초~수분이 소요될 수 있습니다)...


C:\Users\kimjh\anaconda3\envs\ml-dl-nlp\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



산업별 핵심 키워드 추출 결과 (Top 30)
[건설]:
붕괴, 도서관, 매몰, 아파트, 국토부, 치평동, 건축물, 건설업, 모듈러, 하도급, 새만금, 노선, 국토교통부, 수색, 서구, 도로, 교통, 임대, 철골, 콘크리트, 대우건설, 호선, 광주시, 소방, 조물, 시공사, 하청, 중흥, 철근, 감식

[자동차]:
현대차, 자동차, 기아, 전기차, 차량, 주행, 판매, 관세, 수소, 모빌리티, 중국, 타이어, 벤츠, 부품, 테슬라, 판매량, 하이브리드, 버스, 운행, 제네시스, 디스플레이, 전기, 소프트웨어, 연료, 신차, 행차, 로봇, 교통, 경차, 동기

[헬스케어]:
의약품, 신약, 백신, 비만, 셀트리온, 식약처, 투여, 물질, 의료기기, 세포, 임상시험, 질환, 항체, 성분, 판매, 면역, 녹십자, 식품의약품안전처, 체중, 직스, 처방, 스케, 약물, 중국, 품목, 제제, 단백질, 유전자, 유통, 혈당

